In [1]:
import pyodbc
import pandas as pd
import time

In [2]:
def ingest_data_to_sql(csv_file_path, table_name):
    server = 'sqlserver-myotas.database.windows.net'
    database = 'OTAs'
    username = 'azureadmin'
    password = 'brudnyHarry!66'   
    driver = '{ODBC Driver 18 for SQL Server}'
    try:
        cnxn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
        print('Connected')
    except:
        return "Coundn't connect to database"


    # Create a cursor and set fast_executemany to True
    cursor = cnxn.cursor()
    cursor.fast_executemany = True

    # Drop the table if it exists
    start_drop_table = time.time()
    cursor.execute(f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name}")
    end_drop_table = time.time()
    
    # Create the table
    start_create_table = time.time()
    create_table_query = f"CREATE TABLE {table_name} ([City] [nvarchar](255) NULL,\
    [Category] [nvarchar](255) NULL,\
    [MaxPage] [int] NULL,\
    [Date] [date] NULL)"
    cursor.execute(create_table_query)
    end_create_table = time.time()

    # Read CSV file
    data = pd.read_csv(csv_file_path)
    
    # Prepare insert query
    insert_query = f"INSERT INTO {table_name} ([City], [Category], [MaxPage], [Date]) VALUES (?, ?, ?, ?)"
    
    
    data['MaxPage'] = pd.to_numeric(data['MaxPage'], errors='coerce')
    data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
    data = data.drop_duplicates(subset=['City', 'Category'], keep='last')
    data_list = [tuple(row) for row in data.values]
    # Insert data into the table
    start = time.time()
    try:
        cursor.executemany(insert_query, data_list)
        cnxn.commit()
        print(f'Successfully executed. Inserted: {len(data_list)} rows')
    except pyodbc.DataError as e:
        print(e)
        print(e.with_traceback())

    # Close the connection
    cnxn.close()

    # Calculate and print the execution time
    end = time.time()
    print(f"Drop time: {round(end_drop_table - start_drop_table, 4)} Create time: {round(end_create_table - start_create_table, 4)} Execution time: {round(end - start, 4)} seconds")
    return f'Successfully inserted: {len(data_list)} rows to {table_name} table in {round(end - start, 4)} seconds'

# Usage example
# Replace the parameters with your database credentials and CSV file path
gyg_max_page_file = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Resource/GYG_max_page.csv'
viator_max_page_file = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Resource/Viator_max_page.csv'
result = ingest_data_to_sql(gyg_max_page_file, 'GYG_max_page')
print(result)
result = ingest_data_to_sql(viator_max_page_file, 'Viator_max_page')
print(result)

Connected


C:\Users\Wojciech\AppData\Local\Temp\ipykernel_39540\553730800.py:40: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data['Date'] = pd.to_datetime(data['Date'], errors='coerce')


,City,Category,MaxPage,Date
0,Amsterdam,Global,21,2023-11-21
1,Amsterdam,entryTicket,3,2023-11-21
2,Amsterdam,guidedTour,5,2023-11-21
3,Amsterdam,privateTour,6,2023-11-21
4,Amsterdam,other,3,2023-11-21
...,...,...,...,...
1743,Pompeii,127518,8,2023-11-21
1744,Pompeii,4104,8,2023-11-21
1745,Pompeii,128033,7,2023-11-21
1746,Pompeii,128083,8,2023-11-21


[('Amsterdam', 'Global', 21, Timestamp('2023-11-21 00:00:00')), ('Amsterdam', 'entryTicket', 3, Timestamp('2023-11-21 00:00:00')), ('Amsterdam', 'guidedTour', 5, Timestamp('2023-11-21 00:00:00')), ('Amsterdam', 'privateTour', 6, Timestamp('2023-11-21 00:00:00')), ('Amsterdam', 'other', 3, Timestamp('2023-11-21 00:00:00')), ('Amsterdam', 'waterActivity', 3, Timestamp('2023-11-21 00:00:00')), ('Amsterdam', 'dayTrip', 2, Timestamp('2023-11-21 00:00:00')), ('Amsterdam', 'cityCard', 1, Timestamp('2023-11-21 00:00:00')), ('Amsterdam', 'transfer', 2, Timestamp('2023-11-21 00:00:00')), ('Amsterdam', 'bundle', 2, Timestamp('2023-11-21 00:00:00')), ('Amsterdam', 'hopOnHopOff', 1, Timestamp('2023-11-21 00:00:00')), ('Amsterdam', 'workshopOrClass', 1, Timestamp('2023-11-21 00:00:00')), ('Amsterdam', 'adventure', 1, Timestamp('2023-11-21 00:00:00')), ('Amsterdam', 'multiDayTrip', 1, Timestamp('2023-11-21 00:00:00')), ('Amsterdam', 'local_culture', 10, Timestamp('2023-11-21 00:00:00')), ('Amsterdam'

Successfully executed. Inserted: 1744 rows
Drop time: 0.034 Create time: 0.041 Execution time: 0.3367 seconds
Successfully inserted: 1744 rows to GYG_max_page table in 0.3367 seconds
Connected


C:\Users\Wojciech\AppData\Local\Temp\ipykernel_39540\553730800.py:40: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data['Date'] = pd.to_datetime(data['Date'], errors='coerce')


,City,Category,MaxPage,Date
0,Test,Global,10,2023-11-21
1,Lisbon,Sightseeing,84,2023-11-21
2,Lisbon,Tours-by-Duration,82,2023-11-21
3,Lisbon,Culture,71,2023-11-21
4,Lisbon,Historical-and-Heritage-Tours,58,2023-11-21
...,...,...,...,...
73,Porto,Historical-and-Heritage-Tours,14,2023-11-21
74,Lisbon,Port-Transfers,57,2023-11-21
75,Porto,Wine-Tour-Product-Food-Wine-and-Nightlife,8,2023-11-21
76,Porto,Port-Transfers,16,2023-11-21


[('Test', 'Global', 10, Timestamp('2023-11-21 00:00:00')), ('Lisbon', 'Sightseeing', 84, Timestamp('2023-11-21 00:00:00')), ('Lisbon', 'Tours-by-Duration', 82, Timestamp('2023-11-21 00:00:00')), ('Lisbon', 'Culture', 71, Timestamp('2023-11-21 00:00:00')), ('Lisbon', 'Historical-and-Heritage-Tours', 58, Timestamp('2023-11-21 00:00:00')), ('Porto', 'Half-day-Tours', 9, Timestamp('2023-11-21 00:00:00')), ('Porto', 'Bus-and-Minivan-Tours', 18, Timestamp('2023-11-21 00:00:00')), ('Lisbon', 'Half-day-Tours', 25, Timestamp('2023-11-21 00:00:00')), ('Lisbon', 'Luxury-Car-Tours', 25, Timestamp('2023-11-21 00:00:00')), ('Lisbon', 'Luxury-Tours', 22, Timestamp('2023-11-21 00:00:00')), ('Lisbon', 'Transfers-and-Ground-Transport', 21, Timestamp('2023-11-21 00:00:00')), ('Lisbon', 'City-Tours', 20, Timestamp('2023-11-21 00:00:00')), ('Lisbon', 'Classes-and-Workshops', 4, Timestamp('2023-11-21 00:00:00')), ('Lisbon', 'Seasonal-and-Special-Occasions', 14, Timestamp('2023-11-21 00:00:00')), ('Porto', '